##**4. LSTM, GRU**
1. 기존 RNN과 다른 부분에 대해서 배웁니다.
2. 이전 실습에 이어 다양한 적용법을 배웁니다.

### **필요 패키지 import**

In [ ]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

### **데이터 전처리**

아래의 sample data를 확인해봅시다.  
이전 실습과 동일합니다.

In [ ]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [ ]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

100%|██████████| 10/10 [00:00<00:00, 21586.74it/s]

Maximum sequence length: 20


In [ ]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

### **LSTM 사용**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

In [ ]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

In [ ]:
# d_w: word embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)
print(c_n.shape)

PackedSequence(data=tensor([[-8.6145e-02, -7.7185e-02,  1.0082e-01,  ...,  1.8603e-01,
          3.2836e-02,  1.4278e-02],
        [-1.3184e-01,  1.3936e-02, -7.3821e-02,  ...,  2.3812e-04,
          4.8815e-02,  5.4971e-02],
        [ 7.8889e-02,  7.7676e-02,  5.7332e-03,  ..., -9.2923e-02,
          1.2078e-01,  2.5960e-02],
        ...,
        [-3.5228e-02,  4.1591e-02,  1.0066e-01,  ..., -1.4533e-01,
         -8.2826e-02, -2.7983e-01],
        [ 1.4246e-01,  2.2567e-01,  1.8957e-01,  ...,  1.1012e-01,
          3.0236e-02, -2.9021e-02],
        [ 2.0510e-02,  1.0498e-01,  6.7528e-02,  ...,  3.6958e-02,
          4.5707e-02, -7.0221e-03]], grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 512])
torch.Size([1, 10, 512])
torch.Size([1, 10, 512])


In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

torch.Size([20, 10, 512])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [ ]:
# language modeling위해선 전체 시퀀스를 통으로 RNN에 넣지 않고 한 timestep씩 직접 넣어줘야 함. (for loop) - RNN 자체가 전체 단어를 순차적으로 알아서 처리하므로(전체 단어 길이만큼 for문 알아서 돈다) RNN사용할때는 우리가 한 단어씩 늘려서 넣어준다는 의미인듯
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [ ]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [ ]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

Teacher forcing 없이 이전에 얻은 결과를 다음 input으로 이용합니다.
- Teacher forcing : 'I want to go home' -> 모델이 처음에는 엉뚱 (I->want) 등. 근데, 그 효과가 누적될 수 있어 전체 학습이 더디게 될 수 있음. 따라서 처음에는 cheating하도록 설정

In [ ]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

`max_len`만큼의 for 문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(예를 들어 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘 수도 있습니다.

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.

In [ ]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [ ]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[ 0.0956, -0.0023, -0.1301,  ..., -0.0472,  0.1558,  0.0505],
        [-0.0164,  0.0534, -0.0033,  ..., -0.0160,  0.0700, -0.2065],
        [-0.1037,  0.1491,  0.0448,  ...,  0.0566, -0.1261,  0.0292],
        ...,
        [-0.2120,  0.2283,  0.1148,  ..., -0.1311,  0.0294,  0.0169],
        [ 0.0030, -0.0220, -0.2577,  ..., -0.0590, -0.0195,  0.1105],
        [-0.1691, -0.0388, -0.0970,  ..., -0.0098, -0.0255, -0.0033]],
       grad_fn=<CatBackward>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [ ]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[ 0.2195,  0.2885,  0.2273,  ..., -0.3875, -0.4928,  0.4474],
          [-0.2818, -0.2933, -0.0681,  ...,  0.4554,  0.0052,  0.3463],
          [ 0.1916, -0.1246, -0.2031,  ..., -0.3548, -0.2343, -0.4410],
          ...,
          [-0.1806,  0.2979,  0.2052,  ..., -0.2836,  0.1657,  0.4879],
          [-0.2876, -0.5208,  0.0696,  ..., -0.1788,  0.1447, -0.2424],
          [ 0.4693, -0.4728,  0.0316,  ...,  0.4507, -0.3051, -0.1802]],

         [[ 0.1882, -0.3316,  0.1050,  ...,  0.2140,  0.1317,  0.0012],
          [-0.1469, -0.0026,  0.1723,  ..., -0.1883,  0.0883,  0.3696],
          [-0.3605, -0.1945,  0.2039,  ..., -0.1659,  0.0114,  0.0355],
          ...,
          [-0.3506,  0.1695, -0.0071,  ..., -0.2942,  0.2143, -0.0908],
          [ 0.0967,  0.1498,  0.0586,  ..., -0.1027, -0.2590,  0.1201],
          [ 0.1736, -0.0064,  0.1554,  ..., -0.0955, -0.0006,  0.0650]]],


        [[[-0.1691, -0.0388, -0.0970,  ...,  0.0771, -0.2513, -0.0101],
          [-0.1112,  0.1285,